In [8]:
import numpy as np
import cv2 
from time import time
from os.path import join
import random
# from PIL import Image
# from IPython.display import display
# import matplotlib.pyplot as plt
buf = []
def showX(X, rows=1):
    assert X.shape[0]%rows == 0
#     int_X = ( (X+1)/2*255).clip(0,255).astype('uint8')
#     if channel_first:
#         int_X = np.moveaxis(int_X.reshape(-1,3,imageSize,imageSize), 1, 3)
#     else:
#         int_X = int_X.reshape(-1,imageSize,imageSize, 3)
#     int_X = int_X.reshape(rows, -1, imageSize, imageSize,3).swapaxes(1,2).reshape(rows*imageSize,-1, 3)
#     
#     im = im.resize( (loadSize, loadSize), Image.BILINEAR )
    display(Image.fromarray(int_X))
    
def detect_face(image, return_max_area_face=True):
    if len(image.shape) > 2 and image.shape[2] == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        image = cv2.imdecode(image, cv2.CV_LOAD_IMAGE_GRAYSCALE)
    faces = cascade_classifier.detectMultiScale(
        image,
        scaleFactor=1.3,
        minNeighbors=5
    )
    # None is we don't found an image
    if not len(faces) > 0:
        return []
    if return_max_area_face:
        max_area_face = faces[0]
        for face in faces:
            if face[2] * face[3] > max_area_face[2] * max_area_face[3]:
                max_area_face = face
        faces = [max_area_face, ]

    return faces
def bounding_face(frame, faces, factor, dim_constraint=0):
    for face in faces:
        x,y,w,h = face
        factor = 2.
        offset = (factor-1)/2.
        rect = [x-offset*w, y-offset*h, w*factor, h*factor]
        for i,a in enumerate(rect):
            rect[i] = int(a)
        rect = tuple(rect)



        x,y = rect[:2]
        x2 = x + rect[2]
        y2 = y + rect[3]
    #         for 蔡英文
#         print x,x2
        if x > 450 and x2 < 1000:
            cv2.rectangle(frame, (int(x), int(y)), (int(x2), int(y2)), color=(255,255,255), thickness=5)
            a = w*h
            buf.append(a)
            print w,h
    return frame
def bounding_face_keep(frame, faces, box_constraint):
    for face in faces:
        x,y,w,h = face
        offset_x = (box_constraint-w)/2.
        offset_y = (box_constraint-h)/2.
        rect = [x-offset_x, y-offset_y, box_constraint, box_constraint]
        for i,a in enumerate(rect):
            rect[i] = int(a)
        rect = tuple(rect)

        

        x,y = rect[:2]
        x2 = x + rect[2]
        y2 = y + rect[3]
    #         for 蔡英文
        
        if x > 450 and x2 < 1000:
            cv2.rectangle(frame, (int(x), int(y)), (int(x2), int(y2)), color=(255,255,255), thickness=5)
            a = w*h
            buf.append(a)
    return frame


    
CASC_PATH = join('./','haarcascade_files','haarcascade_frontalface_default.xml')
cascade_classifier = cv2.CascadeClassifier(CASC_PATH)

# cap = cv2.VideoCapture(0)q
video_path = join('./','buf_data','520_TW_president_speech.mp4')
cap = cv2.VideoCapture(video_path)
# cap = cv2.VideoCapture(0)
for i in range(random.randint(0,100)):
    ret, frame = cap.read()
print 'start'

while(1):
    
    ret, frame = cap.read()
    faces = detect_face(frame, return_max_area_face=False)
#     faces = detect_face(frame)
#     frame = bounding_face(frame, faces, 2.)
    frame = bounding_face_keep(frame, faces, 280)
    cv2.imshow('frame',frame)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('k'):
        for i in range(random.randint(0,200)):
            ret, frame = cap.read()
        

cap.release()
cv2.destroyAllWindows()
print np.mean(buf), np.var(buf)

start
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
k
22002.618181818183 17298242.49966942


In [2]:
import numpy as np
import cv2 
from time import time
from os.path import join
    
def detect_face(image, return_max_area_face=True):
    if len(image.shape) > 2 and image.shape[2] == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        image = cv2.imdecode(image, cv2.CV_LOAD_IMAGE_GRAYSCALE)
    faces = cascade_classifier.detectMultiScale(
        image,
        scaleFactor=1.3,
        minNeighbors=5
    )
    # None is we don't found an image
    if not len(faces) > 0:
        return []
    if return_max_area_face:
        max_area_face = faces[0]
        for face in faces:
            if face[2] * face[3] > max_area_face[2] * max_area_face[3]:
                max_area_face = face
        faces = [max_area_face, ]

    return faces
def crop_face_keep(frame, faces, box_constraint):
    for face in faces:
        x,y,w,h = face
        offset_x = (box_constraint-w)/2.
        offset_y = (box_constraint-h)/2.
        rect = [x-offset_x, y-offset_y, box_constraint, box_constraint]
        for i,a in enumerate(rect):
            rect[i] = int(a)
        rect = tuple(rect)

        

        x,y = rect[:2]
        x2 = x + rect[2]
        y2 = y + rect[3]
    #         for 蔡英文
        if x > 450 and x2 < 1000:
            cv2.rectangle(frame, (int(x), int(y)), (int(x2), int(y2)), color=(255,255,255), thickness=5)
            return True, frame[y:y2,x:x2,:]
    return False, None


    
CASC_PATH = join('./','haarcascade_files','haarcascade_frontalface_default.xml')
cascade_classifier = cv2.CascadeClassifier(CASC_PATH)

# cap = cv2.VideoCapture(0)q
video_path = join('./','buf_data','520_TW_president_speech.mp4')
cap = cv2.VideoCapture(video_path)
# cap = cv2.VideoCapture(0)
idx = 0
box = 280
    
path = join('./','buf_data','face')
while(1):
    
    ret, frame = cap.read()
    faces = detect_face(frame, return_max_area_face=False)
    crop, face = crop_face_keep(frame, faces, box)
    if crop:
        out_path = join(path,'%d.jpg' % idx)
        idx += 1
        cv2.imwrite(out_path,face)
        for i in range(10):
            ret, frame = cap.read()
    
cap.release()
cv2.destroyAllWindows()
print 'Done'

AttributeError: 'NoneType' object has no attribute 'shape'